# Match movie titles in Movielens and Wikipedia data

In [2]:
import re
import pandas as pd

movies = pd.read_csv("data/ml-20m/movies.csv")

def extract_year(x):
    match = re.search(r'.*\(([0-9]{4})\)', x.title)
    if(match is None):
        year = 0
    else:
        year = match.group(1)
    return int(year)

movies['year'] = movies.apply(extract_year, axis=1)

#movies.year.plot(kind='hist', bins=range(1900, 2020, 5)).set_title("Movies per year")


In [3]:
from fuzzywuzzy import process

def fuzzy_find_matching_index(string, string_list):
    fuzzy_match, similarity_pct = process.extractOne(string, string_list)

    # Boost the similarity of long strings
    if(similarity_pct >= 90):
        similarity_pct += len(string) / 4
        
    if(similarity_pct >= 95): # Take only the ones with high similarity
        print(string, "\t\t", fuzzy_match, "\t\t", similarity_pct)
        return string_list.index(fuzzy_match)
    else:
        return None

def create_title_map_for_year(ml20m_movies, year):
    wiki_plots = pd.read_table("data/wikipedia/wikipedia_plots_%d.csv" % year)
    wiki_titles = wiki_plots.apply(lambda x: x.title + "(%d)" % x.year, axis=1)
    titlemap = ml20m_movies[ml20m_movies.year == year].title.apply(lambda s: fuzzy_find_matching_index(s, list(wiki_titles)))
    titlemap = titlemap.dropna()
    return titlemap.astype(int).to_dict()

In [4]:
#yeardicts = {}
#for y in range(2000, 2016):
#    yeardicts[y] = create_title_map_for_year(movies, y)
pref = pd.read_csv("data/output/pref_vectors.csv")
content = pd.read_csv("data/output/content_vectors.csv")

In [ ]:
wiki_titles = content.title.apply(lambda x: x.decode('utf-8'))
titlemap = movies.title.apply(lambda title: fuzzy_find_matching_index(title, list(wiki_titles)))

In [ ]:
import pickle

with open('titlemap.pickle', 'wb') as f:
    pickle.dump(yeardicts, f)